In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, ReLU
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.regularizers import l2
from keras import applications
import os

from keras.backend import clear_session
clear_session()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16576998068315665102
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5077532672
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4660279814795460594
physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


Using TensorFlow backend.


In [ ]:
train_folders_path = ''
validation_folders_path = ''
test_folders_path = ''

In [2]:
classlabels = ['a','b']

In [3]:
# Image dimensions
img_width, img_height = 224, 224

In [4]:
ResNet50Model = applications.ResNet50(include_top=False, input_shape= (img_width,img_height,3))
ResNet50Model.trainable = False  #Freeze the weights of the transferred model
ResNet50Model.summary()

C:\Users\Ong\Desktop\Programming\Anaconda3\envs\ML\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [5]:
GAP2D = GlobalAveragePooling2D()
PredictionLayer = Dense(units = len(classlabels), activation = 'softmax')
model = Sequential([ResNet50Model,
                    GAP2D,
                    PredictionLayer])

model.compile(optimizer = keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   rotation_range=30,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   validation_split=0.3)

print('Making training data generator...')
training_set = datagen.flow_from_directory('C:/Users/ocw_9/OneDrive/Desktop/Intern/Preprocessed Images/Train',
                                           target_size = (img_width, img_height),
                                           batch_size = 32,
                                           class_mode = 'categorical',
                                           subset='training')

print('Making validation data generator...')
cv_set = datagen.flow_from_directory('C:/Users/ocw_9/OneDrive/Desktop/Intern/Preprocessed Images/Train',
                                       target_size = (img_width, img_height),
                                       batch_size = 32,
                                       class_mode = 'categorical',
                                       subset='validation',
                                       shuffle=False)

In [ ]:
ES1 = EarlyStopping(monitor='val_accuracy', patience=50)
MC1 = ModelCheckpoint(
    filepath=(os.path.join(path, 'ResNet50 with top layers warmed.h5')),
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)
LR1 = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=10, mode='max',verbose=1)

In [ ]:
history = model.fit_generator(training_set,
                         steps_per_epoch = (training_set.n // 32)+1,
                         epochs = 250,
                         validation_data = cv_set,
                         validation_steps = (cv_set.n//32)+1,
                         callbacks=[ES1,MC1,LR1])

In [ ]:
ES2 = EarlyStopping(monitor='val_accuracy', patience=50)
MC2 = ModelCheckpoint(
    filepath=(os.path.join(path, 'ResNet50 Full.h5')),
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)
LR2 = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=20, mode='max',verbose=1)

In [19]:
ResNet50Model.trainable = True
for layer in ResNet50Model.layers[:149]:
    layer.trainable = False

model.compile(optimizer = keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 12,093,442
Non-trainable params: 11,498,368
_________________________________________________________________


In [ ]:
from sklearn.metrics import classification_report

y_pred=np.argmax(classifier.predict_generator(cv_set,cv_set.n // 32+1),axis=1)
print('Classification Report')
print(classification_report(cv_set.classes,y_pred,target_names=classlabels))